In [23]:
import sys
import os
import logging
from kafka import KafkaConsumer
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.functions import col, explode, from_json, udf, when,  abs as spark_abs, array
from pyspark.sql.types import StructType, StructField, StringType
from pyspark.sql.types import ArrayType, MapType, IntegerType, FloatType, BooleanType, DoubleType
import re
import json

In [2]:
spark = SparkSession.builder \
    .appName("KafkaConsumer") \
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.4.0") \
    .getOrCreate()



your 131072x1 screen size is bogus. expect trouble
25/04/28 23:51:51 WARN Utils: Your hostname, LAPTOP-I95CP2I9 resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
25/04/28 23:51:51 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/opt/spark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/tienanh/.ivy2/cache
The jars for the packages stored in: /home/tienanh/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-973edf83-6421-4c25-b40f-13a630459abb;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.4.0 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.4.0 in central
	found org.apache.kafka#kafka-clients;3.3.2 in central
	found org.lz4#lz4-java;1.8.0 in central
	found org.xerial.snappy#snappy-java;1.1.9.1 in central
	found org.slf4j#slf4j-api;2.0.6 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.4 in central
	found org.apache.hadoop#hadoop-client-api;3.3.4 in central
	found commons-logging#commons-logging;1.1.3 in central
	found com.google.code.findbugs#jsr305;3.0.0 in central
	found org.apache.commons#commons-pool2;2.11.1 in central
downloading https://repo1.maven.org/maven2/org/apache/spark/s

In [29]:
parsed_df = spark.read.csv('data/ouput/raw_data.csv', header=True)
parsed_df.show(5)

+--------------------+---------------+------------------------------------------+--------------------+--------------------+--------------------+---------------------------+--------------------+
|        Tên sản phẩm|Loại điện thoại|Màu sắc - Phiên bản bộ nhớ - Giá tương ứng|  Thời gian bảo hành|   Thông số kỹ thuật|            Đánh giá|Số lượt đánh giá và hỏi đáp|           Đường dẫn|
+--------------------+---------------+------------------------------------------+--------------------+--------------------+--------------------+---------------------------+--------------------+
|Điện thoại Xiaomi...|          Redmi|                      [['Xanh Đậm', '4-...|  BH Thường 12 tháng|"{""Màn hình:"": ...| 500 nits (typ)\n...|        HD+ (720 x 1650 ...| ""Hệ điều hành:"...|
|Điện thoại Xiaomi...|          Redmi|                      [['Tím', '4-128GB...|  BH Thường 12 tháng|"{""Màn hình:"": ...|               120Hz|             450 nits (typ)| 600 nits (HBM)\n...|
|Điện thoại Xiaomi...|        

In [30]:
renamed_columns = {
        "Tên sản phẩm": "ten",
        "Loại điện thoại": "loai_dien_thoai",
        "Màu sắc - Phiên bản bộ nhớ - Giá tương ứng": "mau_sac_phien_ban_bo_nho_gia_moi_gia_cu",
        "Thời gian bảo hành": "thoi_gian_bao_hanh",
        "Thông số kỹ thuật": "thong_so_ky_thuat",
        "Đánh giá": "danh_gia",
        "Số lượt đánh giá và hỏi đáp": "so_luong_binh_luan",
        "Đường dẫn": "duong_dan"
    }

for old_name, new_name in renamed_columns.items():
    parsed_df = parsed_df.withColumnRenamed(old_name, new_name)
    
    # Loại bỏ các hàng có giá trị "ten" không mong muốn
parsed_df = parsed_df.filter(col("ten") != "Điện thoại Xiaomi SU7 (Ô tô điện chạy 800km/1 lần sạc)")

    # Loại bỏ các hàng có giá trị "loai_dien_thoai" không mong muốn
parsed_df = parsed_df.filter((col("loai_dien_thoai") != "Apple Watch") & 
                                (col("loai_dien_thoai") != "Máy Chơi Game PC"))
    

In [31]:
parsed_df.show(5)

+--------------------+---------------+---------------------------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|                 ten|loai_dien_thoai|mau_sac_phien_ban_bo_nho_gia_moi_gia_cu|  thoi_gian_bao_hanh|   thong_so_ky_thuat|            danh_gia|  so_luong_binh_luan|           duong_dan|
+--------------------+---------------+---------------------------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|Điện thoại Xiaomi...|          Redmi|                   [['Xanh Đậm', '4-...|  BH Thường 12 tháng|"{""Màn hình:"": ...| 500 nits (typ)\n...| HD+ (720 x 1650 ...| ""Hệ điều hành:"...|
|Điện thoại Xiaomi...|          Redmi|                   [['Tím', '4-128GB...|  BH Thường 12 tháng|"{""Màn hình:"": ...|               120Hz|      450 nits (typ)| 600 nits (HBM)\n...|
|Điện thoại Xiaomi...|          Redmi|                   [['Xanh', '4-128G...|  

In [32]:
parsed_df = parsed_df.withColumn(
    "mau_sac_phien_ban_bo_nho_gia_moi_gia_cu",
    F.from_json(
        F.col("mau_sac_phien_ban_bo_nho_gia_moi_gia_cu"),
        ArrayType(ArrayType(StringType()))  # vì mỗi item là 1 list [mau_sac, phien_ban_bo_nho, gia_moi, gia_cu]
    )
)

# 4. Nổ (explode) từng phần tử của 'mau_sac_phien_ban_bo_nho_gia_moi_gia_cu'
parsed_df = parsed_df.withColumn(
    "exploded",
    F.explode("mau_sac_phien_ban_bo_nho_gia_moi_gia_cu")
)

# 5. Tách các giá trị trong exploded thành các cột riêng biệt
parsed_df = parsed_df.withColumn("mau_sac", F.col("exploded")[0]) \
                      .withColumn("phien_ban_bo_nho", F.col("exploded")[1]) \
                      .withColumn("gia_moi", F.col("exploded")[2]) \
                      .withColumn("gia_cu", F.col("exploded")[3]) \
                      .drop("exploded", "mau_sac_phien_ban_bo_nho_gia_moi_gia_cu")

parsed_df.show(5)

+--------------------+---------------+------------------+--------------------+--------------------+--------------------+--------------------+-----------+----------------+-----------+----------+
|                 ten|loai_dien_thoai|thoi_gian_bao_hanh|   thong_so_ky_thuat|            danh_gia|  so_luong_binh_luan|           duong_dan|    mau_sac|phien_ban_bo_nho|    gia_moi|    gia_cu|
+--------------------+---------------+------------------+--------------------+--------------------+--------------------+--------------------+-----------+----------------+-----------+----------+
|Điện thoại Xiaomi...|          Redmi|BH Thường 12 tháng|"{""Màn hình:"": ...| 500 nits (typ)\n...| HD+ (720 x 1650 ...| ""Hệ điều hành:"...|   Xanh Đậm|          4-64GB|1.650.000 ₫|2.950.000₫|
|Điện thoại Xiaomi...|          Redmi|BH Thường 12 tháng|"{""Màn hình:"": ...| 500 nits (typ)\n...| HD+ (720 x 1650 ...| ""Hệ điều hành:"...|   Xanh Đậm|         4-128GB|1.950.000 ₫|2.950.000₫|
|Điện thoại Xiaomi...|        

In [33]:
parsed_df = parsed_df.withColumn(
    "thong_so_ky_thuat",
    F.from_json(F.col("thong_so_ky_thuat"), ArrayType(StringType()))
)

# 2. Tạo các cột mới từ các phần tử trong array
parsed_df = parsed_df.withColumn("man_hinh",        F.col("thong_so_ky_thuat")[0]) \
                     .withColumn("he_dieu_hanh",     F.col("thong_so_ky_thuat")[1]) \
                     .withColumn("camera_sau",       F.col("thong_so_ky_thuat")[2]) \
                     .withColumn("camera_truoc",     F.col("thong_so_ky_thuat")[3]) \
                     .withColumn("cpu",              F.col("thong_so_ky_thuat")[4]) \
                     .withColumn("ram",              F.col("thong_so_ky_thuat")[5]) \
                     .withColumn("bo_nho_trong",     F.col("thong_so_ky_thuat")[6]) \
                     .withColumn("the_sim",          F.col("thong_so_ky_thuat")[7]) \
                     .withColumn("dung_luong_pin",   F.col("thong_so_ky_thuat")[8]) \
                     .withColumn("thiet_ke",         F.col("thong_so_ky_thuat")[9])

# 3. Xóa cột 'thong_so_ky_thuat' gốc
parsed_df = parsed_df.drop("thong_so_ky_thuat")

parsed_df.show(5)

+--------------------+---------------+------------------+--------------------+--------------------+--------------------+-----------+----------------+-----------+----------+--------+------------+----------+------------+----+----+------------+-------+--------------+--------+
|                 ten|loai_dien_thoai|thoi_gian_bao_hanh|            danh_gia|  so_luong_binh_luan|           duong_dan|    mau_sac|phien_ban_bo_nho|    gia_moi|    gia_cu|man_hinh|he_dieu_hanh|camera_sau|camera_truoc| cpu| ram|bo_nho_trong|the_sim|dung_luong_pin|thiet_ke|
+--------------------+---------------+------------------+--------------------+--------------------+--------------------+-----------+----------------+-----------+----------+--------+------------+----------+------------+----+----+------------+-------+--------------+--------+
|Điện thoại Xiaomi...|          Redmi|BH Thường 12 tháng| 500 nits (typ)\n...| HD+ (720 x 1650 ...| ""Hệ điều hành:"...|   Xanh Đậm|          4-64GB|1.650.000 ₫|2.950.000₫|    NU